# Working Through The Result(s)

In 1.2 we fetched the record(s) through the discovery API and displayed some part of it. Now, we will understand those parts so that we can modify the API call to receive the record we want.

We will start that by displaying the number of records received through the API call.

In [6]:
print(len(response.json()["records"]))

15


In 2.1 we saw that the total number of records (total_results) corresponding to our search is 1608 but the API have returned just 15 records. That is because the default result page size for the API is 15. To change the number of records we receive from the API, we add the query parameter "&sps.resultsPageSize=30" to our URL if we want 30 record. The number should be in the range between 0 and 1000.

In [10]:
result_size = "&sps.resultsPageSize=30"

Now, if we display the number of records received through the API call

In [12]:
full_search_url = base_discovery_url + search_endpoint + "?" + search_query_parameter + "=" + search_query + result_size

response = requests.request("GET", full_search_url)

print(len(response.json()["records"]))

30


If we want to check how many page(s) of records there are

In [13]:
import math
number_of_pages = math.ceil(int(response.json()["count"])/len(response.json()["records"]))
print(number_of_pages)

54


If we want to get all the results, we will add a new query parameter "&sps.page=page_number" and iterate it over the number of pages.

In [16]:
page_number = 2
result_page =  "&sps.page=" + str(page_number)

In [15]:
full_search_url = base_discovery_url + search_endpoint + "?" + search_query_parameter + "=" + search_query + result_size + result_page

response = requests.request("GET", full_search_url)

## First, make sure we got a 200 response

print(response.status_code)

200


Now, we get the records on the 2nd page

In [17]:
print(json.dumps(response.json(), indent=4))

{
    "records": [
        {
            "altName": "",
            "places": [],
            "corpBodies": [],
            "taxonomies": [],
            "formerReferenceDep": "",
            "formerReferencePro": "",
            "heldBy": [
                "Devon Archives and Local Studies Service (South West Heritage Trust)"
            ],
            "context": "Devon Crew Lists.",
            "content": "",
            "urlParameters": null,
            "department": "",
            "note": "",
            "adminHistory": "",
            "arrangement": "",
            "mapDesignation": "",
            "mapScale": "",
            "physicalCondition": "",
            "catalogueLevel": 9,
            "openingDate": "",
            "closureStatus": "",
            "closureType": "",
            "closureCode": "",
            "documentType": "",
            "coveringDates": "1863",
            "description": "",
            "endDate": "31/12/1863",
            "numEndDate": 18631231,
  

We will define a new list in which we will store all the records

In [18]:
result_array = []

And then fetch results for all the pages

In [20]:
for i in range(number_of_pages):
  result_page =  "&sps.page=" + str(i)
  full_search_url = base_discovery_url + search_endpoint + "?" + search_query_parameter + "=" + search_query + result_size + result_page
  response = requests.request("GET", full_search_url)
  result_array = result_array + response.json()["records"]

print(len(result_array))


1620


Now that we have all the record(s) for our query, we will interogate one such record to check what extra information it holds. To do this, we will take the id from our result and use a new API endpoint "/records/v1/details/{id}"

In [21]:
record_id = result_array[1]["id"]
print(record_id)

C13444469


In [24]:
record_endpoint = "/records/v1/details/"

Now, we will fetch the details for that particular record ID

In [25]:
full_search_url = base_discovery_url + record_endpoint + record_id

response = requests.request("GET", full_search_url)

print(json.dumps(response.json(), indent=4))

{
    "accessRegulation": null,
    "accruals": null,
    "accumulationDates": null,
    "administrativeBackground": null,
    "appraisalInformation": null,
    "arrangement": null,
    "batchId": null,
    "copiesInformation": [],
    "corporateNames": [],
    "creatorName": [],
    "custodialHistory": null,
    "detailedRelatedMaterial": [],
    "detailedSeparatedMaterial": [],
    "dimensions": null,
    "formerReferenceDep": null,
    "formerReferencePro": null,
    "immediateSourceOfAcquisition": [],
    "language": "English",
    "legalStatus": null,
    "links": [],
    "locationOfOriginals": [],
    "mapDesignation": null,
    "mapScaleNumber": 0,
    "note": null,
    "otherReferences": [],
    "people": [],
    "physicalCondition": null,
    "physicalDescriptionExtent": null,
    "physicalDescriptionForm": null,
    "places": [],
    "publicationNote": [],
    "registryUrl": "",
    "restrictionsOnUse": null,
    "scannedLists": [],
    "subjects": [],
    "unpublishedFinding

Most of these details might not be useful for us. So, we will create a new list of all the records, but with selected informations like title, description, record open date, and parent ID.

In [26]:
new_result_array = {}

In [27]:
for i in result_array:
  record_id = i["id"]
  full_search_url = base_discovery_url + record_endpoint + record_id
  response = requests.request("GET", full_search_url)
  new_result_array[response.json()["id"]] = {"title": response.json()["title"], "description": response.json()["scopeContent"]["description"], "openingDate": response.json()["recordOpeningDate"], "parentID": response.json()["parentId"]}

KeyboardInterrupt: ignored

Now we display the record in the new list corresponding to the id of the 1st record in the old list and then the description of it.

In [28]:
print(new_result_array[result_array[1]["id"]])
print(new_result_array[result_array[1]["id"]]["description"])

{'title': None, 'description': '<scopecontent><p>HMS Ajax: 13 December 1939: damaged by enemy action, South Atlantic. 1 June 1940: crew lent to Operation Dynamo. 12 October 1940: damaged by enemy action, in action against Italian destroyers. 7 April 1941: damaged by enemy action, air attack. 28 May 1941: damaged by enemy action in the Mediterranean or Middle East. 1 January 1943: damaged by enemy action, air attack in the Mediterranean</p></scopecontent>', 'openingDate': '28/06/2013 00:34:31', 'parentID': 'C16485'}
<scopecontent><p>HMS Ajax: 13 December 1939: damaged by enemy action, South Atlantic. 1 June 1940: crew lent to Operation Dynamo. 12 October 1940: damaged by enemy action, in action against Italian destroyers. 7 April 1941: damaged by enemy action, air attack. 28 May 1941: damaged by enemy action in the Mediterranean or Middle East. 1 January 1943: damaged by enemy action, air attack in the Mediterranean</p></scopecontent>
